In [44]:
from langchain.llms import OpenAI
from langchain_community.chat_models.openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

True

In [45]:
llm=ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],model_name='gpt-3.5-turbo',temperature=0.6)

In [46]:
RESPONSE_JSON_STRUCTURE = {
  "1": {
    "pergunta": "Sua pergunta sobre o conceito do texto",
    "resposta": "A resposta para a sua pergunta"
  },
  "2": {
    "pergunta": "Sua pergunta sobre outro conceito",
    "resposta": "A resposta para a sua segunda pergunta"
  },
  # // ... e assim por diante para cada conceito identificado
}


In [47]:
prompt_template="""
Texto Fornecido: {text}
Como especialista na criação de flashcards educacionais, sua tarefa é analisar o texto fornecido e gerar um conjunto de flashcards que capturem efetivamente seus conceitos-chave e facilitem o aprendizado.
Se o texto fornecido estiver vazio: Você pode retornar uma resposta vazia ou indicar a ausência de conteúdo, mantendo a formatação em JSON, indicada.
Para texto não vazio:
1. Identifique os principais conceitos ou pontos-chave dentro do texto.
2. Para cada conceito identificado, formule uma pergunta objetiva que se relacione diretamente com a informação central desse conceito.
3. Forneça uma resposta concisa e informativa que aborde com precisão a pergunta feita.
4. Retorne os flashcards gerados no seguinte formato JSON:
{response_json}
Observação: Substitua os espaços reservados na estrutura de resposta pelas perguntas e respostas reais derivadas do texto.
"""

In [48]:
card_generation_prompt=PromptTemplate(
    input_variables=['text','response_json'],
    template=prompt_template
)

card_chain=LLMChain(llm=llm,prompt=card_generation_prompt, output_key='card',verbose=True)

In [49]:
#How to setup Token Usage Tracking in LangChain
#with get_openai_callback() as cb:
    #response = card_chain({'text': text, 'response_json': json.dumps(RESPONSE_JSON_STRUCTURE)})


In [50]:
#print(f"Total Tokens:{cb.total_tokens}")
#print(f"Prompt Tokens:{cb.prompt_tokens}")
#print(f"Completion Tokens:{cb.completion_tokens}")
#print(f"Total Cost:{cb.total_cost}")

In [51]:
# Loading PDF files
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter

In [52]:
def extract_PDF_data(pdf_path):
    base_name=os.path.basename(pdf_path)
    pdf_title=os.path.splitext(base_name)[0]
    pdf_title = pdf_title.replace(' ', '_').replace(':', '').replace('/', '_')
    pdf_reader = PdfReader(pdf_path)
    return pdf_reader, pdf_title



In [53]:
pdf_path=r'D:\ProjectsAI\flashcard-generator\data\curso-274931-aula-03-prof-stefan-fantini-2f80-simplificado.pdf'
pdf_reader, pdf_title=extract_PDF_data(pdf_path)

In [54]:
raw_text = ''
# Start from page 4 (index 3) instead of the first page
for i, page in enumerate(pdf_reader.pages):
    if i < 4:  # Skip the first three pages
        continue
    
    content = page.extract_text()
    if content:
        if "RESUMO ESTRATÉGICO" in content:
            break  # Stop importing pages when this text is found
        raw_text += content


In [55]:
print(raw_text)

     FUNÇÃO DIREÇÃO  
A direção  é uma das quatro funções administrativas  que compõe o processo organizacional. Trata -
se da função responsável por conduzir as atividades  das pessoas em direção aos objetivos 
organizacionais.  
A direção está intimamente ligada ao relacionamento interpessoal  entre o administrador e seus 
subordinado s. Chiavenato1 destaca que, “para que o plane jamento e a organização possam ser 
eficazes, eles precisam ser complementados pela orientação e apoio às pessoas , por meio de uma 
adequada comunicação , liderança  e motivação . Para dirigir as pessoas, o administ rador precisa 
saber comunicar, liderar e motivar. Enquanto as outras funções administrativas – planejamento, 
organização e controle – são impessoais, a direção constitui um processo interpessoal  que 
determina relações e interações entre indivíduos.”  
Vejamo s algumas das ações que são desenvolvidas nessa função:  
 
 
1 - Abrangência da Função Direção  
A função direção também está presente

In [56]:
text_splitter=CharacterTextSplitter(
    separator='\n',
    chunk_size=2500,
    length_function=len,
)
text_chunks=text_splitter.split_text(raw_text)

In [57]:
len(text_chunks)

29

In [ ]:
csv_splitter=CharacterTextSplitter(
    separator='\n',
    chunk_size=2500,
    length_function=len,
)
with open(r'D:\ProjectsAI\flashcard-generator\research\curso-274931-aula-03-prof-stefan-fantini-2f80-simplificado.csv', 'r') as csv:
    cards_revised=''
    for chunk in csv_splitter.split_text(csv.read()):
        review=review_chain({'csv_file': json.dumps(chunk)})
        cards_revised.append(review)
    print(cards_revised)


In [58]:
def card_generator(text_chunks,filename):
    card_table_data = []
    
    for chunk in text_chunks:
        response = card_chain({'text': chunk, 'response_json': json.dumps(RESPONSE_JSON_STRUCTURE)})
        card_str = response.get('card')
        
    #for chunk in range(0, 15):
        #response = card_chain({'text': text_chunks[chunk], 'response_json': json.dumps(RESPONSE_JSON_STRUCTURE)})
        #card_str = response.get('card')

        # Try-except block to catch JSON parsing errors
        try:
            card_dict = json.loads(card_str)
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON from chunk: {e}")
            continue  # Skip this chunk and proceed with the next one

        for key, value in card_dict.items():
            pergunta = value.get('pergunta')
            resposta = value.get('resposta')
            if pergunta and resposta:  # Ensure both pergunta and resposta are not None
                card_table_data.append({'pergunta': pergunta, 'resposta': resposta})
            else:
                print(f"Missing 'pergunta' or 'resposta' in chunk: {chunk}")

    df_cards = pd.DataFrame(card_table_data)
    df_cards.to_csv(f'{filename}.csv', index=False)
    

In [ ]:
card_generator(text_chunks,pdf_title)

In [ ]:
review_template="""
Como especialista em educação e processos de aprendizagem, solicito a revisão de flashcards contidos em um arquivo CSV. 
Por favor, analise o conteúdo para identificar e remover quaisquer informações repetitivas ou incoerentes,assegurando assim a eficácia do material como ferramenta de estudo.
"""

review_assistant_prompt=PromptTemplate(
    input_variables=['csv_file'],
    template=prompt_template
)

review_chain=LLMChain(llm=llm,prompt=review_assistant_prompt, verbose=True)

In [ ]:
csv_splitter=CharacterTextSplitter(
    separator='\n',
    chunk_size=2500,
    length_function=len,
)
with open(r'D:\ProjectsAI\flashcard-generator\research\curso-274931-aula-03-prof-stefan-fantini-2f80-simplificado.csv', 'r') as csv:
    cards_revised=''
    for chunk in csv_splitter.split_text(csv.read()):
        review=review_chain({'csv_file': json.dumps(chunk)})
        cards_revised.append(review)
    print(cards_revised)


In [60]:
import csv
import json
import requests

In [61]:
# Path to your CSV file
csv_file_path = r'D:\ProjectsAI\flashcard-generator\research\curso-274931-aula-02-prof-stefan-fantini-236c-simplificado.csv'

# Function to add a note to Anki
def add_note_to_anki(front, back):
    payload = {
        "action": "addNote",
        "version": 6,
        "params": {
            "note": {
                "deckName": "Gestão e Governança Pública",
                "modelName": "Basic",
                "fields": {
                    "Frente": front,
                    "Verso": back
                },
                "options": {
                    "allowDuplicate": False,
                },
                "tags": [
                    "gestão", "governança", "pública"
                ]
            }
        }
    }
    response = requests.post('http://localhost:8765', json=payload)
    return response.json()

# Read the CSV file and add each row as a note in Anki
with open(csv_file_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        front, back = row
        result = add_note_to_anki(front, back)
        print(result)


ConnectionError: HTTPConnectionPool(host='localhost', port=8765): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001538C831C30>: Failed to establish a new connection: [WinError 10061] Nenhuma conexão pôde ser feita porque a máquina de destino as recusou ativamente'))